In [5]:
import pandas as pd
import numpy as np
import re
import requests

In [6]:
def get_current_prices():
    urls = ['http://finance.daum.net/xml/xmlallpanel.daum?stype=P&type=L',
            'http://finance.daum.net/xml/xmlallpanel.daum?stype=Q&type=L']

    stocks = {}
    for url in urls:
        text = requests.get(url).text
        rep_code = re.compile(
            "code.*:\"(.+)\",.*name.*:\"(.+)\",.*cost.*:\"(.+)\",.*updn.*:\"(.+)\",.*rate.*:\"(.+)\"")

        for line in text.splitlines():
            try:
                match = rep_code.search(line)
                code = match.group(1)
                name = match.group(2)
                price = int(match.group(3).replace(',', ''))
                updn = int(match.group(4).replace('▼', '-').replace('▲', '').replace(',', '').replace('&nbsp;', '').replace('↓',''))
                rate = float(match.group(5).replace('%', '').replace('+', ''))
            except AttributeError:  # No match
                continue
            stocks[code] = (code, name, price, updn, rate)

    stocks = pd.DataFrame(stocks).T
    stocks.columns = ['code','name', 'price', 'updn', 'rate']
    # stocks.index.names = ['code']
    # stocks.to_sql('stock_min', conn, if_exists='replace')
    return stocks

In [9]:
df = get_current_prices()
df.head()

,code,name,price,updn,rate
000020,000020,동화약품,10650,0,0
000030,000030,우리은행,16900,50,0.3
000040,000040,KR모터스,581,-9,-1.53
000050,000050,경방,12950,50,0.39
000060,000060,메리츠화재,18550,-100,-0.54


,code,name,price,updn,rate
000020,000020,동화약품,10600,-50,-0.47
000030,000030,우리은행,16850,0,0
000040,000040,KR모터스,582,-8,-1.36
000050,000050,경방,12950,50,0.39
000060,000060,메리츠화재,18550,-100,-0.54
